# O-D Cities Recommender System
This notebook proposes a method to recommend Origin-Destination city routes for cargo drivers. 

## Table of Contents:
* 1 Introduction
    * Background
    * Method
    * Data
    * Summary of Results
* 2 Code
    * Load Data
    * Preprocess
    * {Checkpoint}
    * Build Model
    * Validate
* 3 Conclusion
* 4 Appendix

# 1 Introduction
## Background
A goal of our research group has been to identify cargo-delivery routes. By identifying cargo-delivery routes, we would be able to do higher-order analysis on driver behaviors such as route predictions. 

An approach I thought of is to predict a driver's **preferred** origin-destination (O-D) cities routes. While we lack ground-truth on the specific cargo routes, we may be able to detect the city-city preferences of drivers using the company's "active" cargo data.

The approach is similar to recommender systems carried out by social media platforms such as Netflix. 


## Method
The Recommender System approach used here is **Collaborative Filtering**, which has been commonly used by social media platforms. The specific algorithm is a form of **Matrix Factorization**, which is the **model-based implementation** of Collaborative Filtering. The algorithm is based on the paper titled [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf) by Hu et al. (2008). 

To implement this algorithm, I used the python library [*implicit*](https://github.com/benfred/implicit).  


## Data
The data used was dr250_active.csv.


## Summary of Results
Definitions:
* N: number of OD pairs recommended to each user,
* k: number of OD pairs in the test set for each user.

Using parameters N=100 and k=20, the Collaborative Filtering model scored, using [*recall*](https://en.wikipedia.org/wiki/Precision_and_recall): **~20%**. 

This is compared to the benchmark score of **~0.6%**, based on recommending the same set of top N most popular items to every user.

-----

# 2 Code
Below shows the code and relevant output. The helper script cargo_recommender.py is uploaded in the Google Drive. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [25]:
import pandas as pd
import numpy as np
import implicit
import time
from cargo_recommender import (get_user_item_matrix,
                               train_test_split,
                               validate)

## Load Data

In [7]:
df = pd.read_csv('../data/dr250_active.csv')
print(df.head())

              datetime     userid acttype orgprov orgcity destprov destcity  \
0  2018-03-06 00:42:41  1_1009235  search  yunnan    yuxi      NaN      NaN   
1  2018-03-06 00:42:41  1_1009235  search  yunnan    yuxi      NaN      NaN   
2  2018-03-06 00:42:41  1_1009235  search  yunnan    yuxi      NaN      NaN   
3  2018-03-06 00:42:41  1_1009235  search  yunnan    yuxi      NaN      NaN   
4  2018-03-06 00:42:41  1_1009235  search  yunnan    yuxi      NaN      NaN   

   trucktype  trucklen       orderid  
0         -1        -1  8.379983e+09  
1         -1        -1  8.383683e+09  
2         -1        -1  8.374969e+09  
3         -1        -1  8.385825e+09  
4         -1        -1  8.385566e+09  


## Preprocess
The data is split to train and test using a form of "leave-k-out" approach. 
The procedure is:
1. Split users to train and test sets. 
(Additional requirement for users in test set: must have at least k*10 unique O-D pairs.)
2. For each user in test set, remove k items out of the train set and place in test set.

In [19]:
s = time.time()
user_items = get_user_item_matrix(df)
k = 20
user_items_train, test_users = train_test_split(user_items,
                                               split_method='leave_k_out',
                                               k=k)
print(f'Time taken to preprocess data: {time.time()-s:.2f}s')

Time taken to preprocess data: 0.35s


## {Checkpoint}
Before we build the model, let's examine a few key statistics.
1. Number of unique O-D pairs.
2. Number of unique O-D pairs per user.

## Build Model
The model is built using the [implicit](https://github.com/benfred/implicit) library. The algorithm is based on the paper [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf) by Hu et al (2008). The parameters can be tuned further to improve accuracy. 

In [20]:
s = time.time()
model = implicit.als.AlternatingLeastSquares(factors=10,
                                            regularization=0.1,
                                            iterations=30)
alpha=2
item_users_train = user_items_train.transpose()
model.fit(item_users_train*alpha)
print(f'Time taken to build model: {time.time()-s:.2f}s')

100%|██████████| 30.0/30 [00:01<00:00, 28.69it/s]

Time taken to build model: 1.05s


## Validate
The accuracy was calculated using recall metrics. The formula is, for each user in test set:
    
    (# items in predictions AND test set) / (# items in test set)
    
The accuracy was averaged across all users.

In [22]:
s = time.time()
N = 100
validate(model,user_items,user_items_train,test_users,N=N)
print(f'\nTime taken to validate model: {time.time()-s:.2f}s')


Accuracy (Recall)
----------
Model: 16.765%
Benchmark: 0.686%
[Benchmark based on recommending the top N most popular O-D routes to every user. N = 100]

Time taken to validate model: 0.09s


-----

# 3 Conclusion
The accuracy based on recall is shown above at around 17%. This is not high, presumably because of the high number of unique O-D combinations in the data. Compared to the benchmark however, it has a significant improvement. 


-----

# 4 Appendix

## Further Reading.
The code was adapted from the following two blogs:
* https://towardsdatascience.com/recommending-github-repositories-with-google-bigquery-and-the-implicit-library-e6cce666c77
* https://jessesw.com/Rec-System/